NOTE: I rewrite various notebooks because that's how I learn. I do it on Kaggle because I like their community and other features. Please use and credit original source.

Source: https://github.com/tensorflow/docs/blob/master/site/en/r2/guide/autograph.ipynb

In [1]:
!pip install -U tensorflow==2.0.0a0

    100% |████████████████████████████████| 79.9MB 487kB/s eta 0:00:01    50% |████████████████▎               | 40.6MB 20.7MB/s eta 0:00:02    55% |█████████████████▋              | 44.0MB 19.8MB/s eta 0:00:02    58% |██████████████████▋             | 46.4MB 30.4MB/s eta 0:00:02    94% |██████████████████████████████  | 75.1MB 27.6MB/s eta 0:00:01    99% |███████████████████████████████▊| 79.3MB 29.4MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 20.6MB/s ta 0:00:01
    100% |████████████████████████████████| 3.0MB 17.6MB/s ta 0:00:01
    100% |████████████████████████████████| 419kB 31.1MB/s ta 0:00:01
  Found existing installation: tensorflow 1.12.0
    Uninstalling tensorflow-1.12.0:
      Successfully uninstalled tensorflow-1.12.0
You are using pip version 18.1, however version 21.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Decorators https://www.python.org/dev/peps/pep-0318/

# @dec2
# @dec1
# def func(arg1, arg2, ...):
#     pass

# # This is equivalent to:
# def func(arg1, arg2, ...):
#     pass
# func = dec2(dec1(func))

# tf.function and AutoGraph in TensorFlow 2.0

TF 2.0 bring together the ease of eager execution and the power of TF 1.0. At the center of this merger is `tf.function` which allows you to transform a subset of Python syntax into protable, high-performance TensorFlow graphs.

A cool new feature of `tf.function` is AutoGraph, which lets you write graph code using natural Python syntax. For a list of the Python features that you can use with AutoGraph, see [AutoGraph capabilities and limitations](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/LIMITATIONS.md). For more details aabout `tf.function` see the RFC [TF 2.0: Functions, not Sessions](https://github.com/tensorflow/community/blob/master/rfcs/20180918-functions-not-sessions-20.md). For more details about Autograph, see [tf.autograph](https://www.tensorflow.org/api_docs/python/tf/autograph).

This tutorial will walk you through the basic features of `tf.function` and AutoGraph.

## Setup

Import TensorFlow and enable TF 2.0 mode:

In [3]:
import sys
print("Python version:", sys.version)

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

Python version: 3.6.6 |Anaconda, Inc.| (default, Oct  9 2018, 12:34:16) 
[GCC 7.3.0]
TensorFlow version: 2.0.0-alpha0


In [4]:
from __future__ import absolute_import, division, print_function
import numpy as np

import tensorflow as tf

Install aa temporary patch to enable a few extra TF 2.0 upgrades. This piece will be removed soon.

In [5]:
from tensorflow.python.ops import control_flow_util
control_flow_util.ENABLE_CONTROL_FLOW_V2 = True

## The `tf.function` decorator

When you annotate a function with `tf.function`, you can still call it like any other function. But it will be compiled into a graph, which means you get the benefits of faster execution, running on GPU or TPU, or exporting to SavedModel.

In [6]:
@tf.function
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))

x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

<tf.Tensor: id=25, shape=(3, 3), dtype=float32, numpy=
array([[0.73770696, 0.6176839 , 0.7776219 ],
       [0.68008655, 0.8703463 , 0.9625932 ],
       [0.59634304, 0.6427193 , 0.74688315]], dtype=float32)>

If we examine the result of the annotation, we can see that it's a special callable that handles all interactions with the TensorFlow runtime.

In [7]:
simple_nn_layer

If your code uses multiple functions, you don't need to annotate them all - any functions called from an annotated function will also run in graph mode.

In [8]:
def linear_layer(x):
    return 2 * x + 1

@tf.function
def deep_net(x):
    return tf.nn.relu(linear_layer(x))

deep_net(tf.constant((1, 2, 3)))

<tf.Tensor: id=39, shape=(3,), dtype=int32, numpy=array([3, 5, 7], dtype=int32)>

## Use Python control flow

When using data-dependent control flow inside `tf.function`, you can use Python control flow statements and AutoGraph will convert them into appropriate TensorFlow ops. For example, `if` statements will be converted into `tf.cond()` if they depend on a `Tensor`.

In the example below, x is a `Tensor` but the `if` statement works as expected:

In [9]:
@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x

print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

square_if_positive(2) = 4
square_if_positive(-2) = 0


Note: the example above shows how to perform simple conditionals when scalar values are involved. Typical ML code involves batches; in those cases you should consider using the faster and vecotrized `tf.where` if possible.

AutoGraph supports common Python statements like `while`, `for`, `if`, `break`, `continue`, `return`, with support for nesting. That means you  can use `Tensor` expressions in the condition of `while` and `if` statements, or iterate over a `Tensor` in a `for` loop.

In [10]:
@tf.function
def sum_even(items):
    s = 0
    for c in items:
        if c % 2 > 0:
            continue
        s += c
    return s

sum_even(tf.constant([10, 12, 15, 20]))

<tf.Tensor: id=149, shape=(), dtype=int32, numpy=42>

AutoGraph also provides a low-level API for advanced users. For example, we can use it to have a look at the generated code.

In [12]:
print(tf.autograph.to_code(sum_even.python_function, experimental_optional_features=None))

from __future__ import print_function

def tf__sum_even(items):
  do_return = False
  retval_ = None
  s = 0

  def loop_body(loop_vars, s_2):
    c = loop_vars
    continue_ = False
    cond = c % 2 > 0

    def if_true():
      continue_ = True
      return continue_

    def if_false():
      return continue_
    continue_ = ag__.if_stmt(cond, if_true, if_false)
    cond_1 = ag__.not_(continue_)

    def if_true_1():
      s_1, = s_2,
      s_1 += c
      return s_1

    def if_false_1():
      return s_2
    s_2 = ag__.if_stmt(cond_1, if_true_1, if_false_1)
    return s_2,
  s, = ag__.for_stmt(items, None, loop_body, (s,))
  do_return = True
  retval_ = s
  return retval_



tf__sum_even.autograph_info__ = {}



Here's an example of more complicated control flow:

In [13]:
@tf.function
def fizzbuzz(n):
    msg = tf.constant('')
    for i in range(n):
        if i % 3 == 0:
            msg += 'Fizz'
        elif i % 5 == 0:
            msg += 'Buzz'
        else:
            msg += tf.as_string(i)
        msg += '\n'
    return msg

print(fizzbuzz(tf.constant(15)).numpy().decode())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14



## Use Python print

AutoGraph will also convert Python builtins like `print`.

Note: due to the parallel nature of calculations in TensorFlow, statements might execute out of order. It's best to use `print` only to inspect actual values, and you should not use it to determine whether the program execution reaches a certain point. (What to use for that?)

In [14]:
@tf.function
def count(n):
    for i in tf.range(n):
        print(i)
        
count(tf.constant(5))

Tensor("strided_slice_2:0", shape=(), dtype=int32)


## Other handy conversions

Other builtins that AutoGraph can adapt for TensorFlow are `range` and `len`.

`range` is a shortcut for `tf.range`:

In [15]:
@tf.function
def range_example(n):
    return range(n)

print(range_example(tf.constant(3)))

tf.Tensor([0 1 2], shape=(3,), dtype=int32)


`len` is a shortcut for `.shape[0]`:

In [16]:
@tf.function
def len_example(n):
    return len(n)

print(len_example(tf.zeros((20, 10))))

tf.Tensor(20, shape=(), dtype=int32)


## Kears and AutoGraph

You can use `tf.function` with object methods as well. For example, you can decorate your custom Keras models, typically by annotating the model's `call` function. For more information, see `tf.keras`.

In [17]:
class CustomModel(tf.keras.models.Model):
    
    @tf.function
    def call(self, input_data):
        if tf.reduce_mean(input_data) > 0:
            return input_data
        else:
            return input_data // 2
        
model = CustomModel()
model(tf.constant([-2, -4]))

<tf.Tensor: id=335, shape=(2,), dtype=int32, numpy=array([-1, -2], dtype=int32)>

## Side effects

Just like in eager mode, you can use operations with side effects, like `tf.assign` or `tf.print` normally inside `tf.function`, and it will insert the necessary control dependencies to ensure they execute in order.

In [18]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
    v.assign(v + 1)
    if v % 2 == 0:
        v.assign(v + 1)

find_next_odd()
v

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=6>

## Example: training a simple model

AutoGraph also allows you to mode more compuration inside TensorFlow, For example, a training loop is just control flow, so it can actually be brought into TensorFlow.

### Download data

In [19]:
def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    
    return x, y

def mnist_dataset():
    (x, y), _ = tf.keras.datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

train_dataset = mnist_dataset()

11493376/11490434 [==============================] - 0s 0us/step


### Define the model

In [20]:
model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))

model.build()
optimizer = tf.keras.optimizers.Adam()

### Define the training loop

In [21]:
def compute_loss(logits, labels):
    return tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels))

def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(predictions == labels, tf.float32))

def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        tape.watch(model.variables)
        logits =  model(x)
        loss = compute_loss(logits, y)
        
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(zip(grads, model.variables))
    
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy


@tf.function
def train(model, optimizer):
    train_ds = mnist_dataset()
    step = 0
    for x, y in train_ds:
        step += 1
        loss, accuracy = train_one_step(model, optimizer, x, y)
        if step % 10 == 0:
            print('Step', step, ': loss', loss, ':, accuracy', accuracy)
    return step

_ =  train(model, optimizer)

## A note on batching

In real applications batching is essential for performance. The best code to convert to AutoGraph is code where the control flow is decided at the *batch* level. If making decisions at the individual *example* level, try to use batch APIs to maintain performance.

For example, if you have the following code in Python:

In [22]:
def square_if_positive(x):
    return [i ** 2 if i > 0 else i for i in x]

square_if_positive(range(-5, 5))

[-5, -4, -3, -2, -1, 0, 1, 4, 9, 16]

You may be tempted to write it in TensorFlow as such (and this would work!):

In [23]:
@tf.function
def square_if_positive_naive(x):
    result = tf.TensorArray(tf.int32, size=len(x))
    for i in range(len(x)):
        if x[i] > 0:
            result = result.write(i, x[i] ** 2)
        else:
            result = result.write(i, x[i])
    return result.stack()

square_if_positive_naive(tf.range(-5, 5))

TypeError: object of type 'Tensor' has no len()

But in this case, it turns out you can write the following:

In [24]:
def square_if_positive_vectorized(x):
    return tf.where(x > 0, x ** 22, x)

square_if_positive_vectorized(tf.range(-5, 5))

<tf.Tensor: id=1297, shape=(10,), dtype=int32, numpy=
array([        -5,         -4,         -3,         -2,         -1,
                0,          1,    4194304, 1316288537,          0],
      dtype=int32)>